# Week 1 Tutorial Notebook: Unix, Compilation, and Performance
*Generated 2026-01-11*




## Purpose

This assignment introduces you to the Unix-based computational environment used throughout the course and in real scientific workflows. You will practice logging into a remote server, compiling programs, running them, and comparing performance across languages.

You are encouraged to use GenAI tools (e.g., ChatGPT) to help write code, and look up Linux command syntax as needed. The goal is understanding workflows and concepts, **not** memorizing syntax.

> **Important:** Many steps below are *terminal commands* meant to be run on your local computer or on the remote server (`newton`). In a notebook, they are shown for copy/paste convenience.



---

## Section A: Server Access & Unix Basics

Here we log on to the server, change your password, and make a directory.

### A1. SSH into the course server

Run this on your **local** computer in a terminal:

```bash
ssh [username]@newton.rc.usf.edu
```

### A2. Change your password

On `newton`:

```bash
passwd
```

Follow the prompts.

### A3. Create a course directory in your home directory

```bash
mkdir -p ~/compsci2026
cd ~/compsci2026
pwd
```

### A4. Log out

```bash
exit
```

**Checkpoint:** You should be able to SSH in/out cleanly, and `~/compsci2026` should exist on `newton`.


---
## Section B: Hello World in Three Languages



Here we write and run a minimal *Hello, world* program in three languages. We copy these onto the server to (compile and) run them, to mimic a common workflow where you write code on your local computer, and then upload it to HPC infrastructure to run it.

### B1. Create the three source files locally

On your **local** computer (not on the server), open Visual Studio Code and create these files:

- `hello.c`
- `hello.f90`
- `hello.py`

You may use ChatGPT or other GenAI tools if needed. Study the code for each. What do you notice?

Below are minimal versions you can copy/paste.


### hello.c
```c
#include <stdio.h>

int main(void) {
    printf("Hello, world from C\n");
    return 0;
}
```


### hello.f90
```fortran
program hello
  implicit none
  print *, "Hello, world from Fortran"
end program hello
```


### hello.py
```python
print("Hello, world from Python")
```



### B2. Copy the files to `newton`

Using `scp` from your **local** terminal (adjust the path if needed):

```bash
scp hello.c hello.f90 hello.py [username]@newton.rc.usf.edu:~/compsci2026/
```

Alternatively, you may use FileZilla.

### B3. SSH to `newton` and compile/run

SSH in:

```bash
ssh [username]@newton.rc.usf.edu
cd ~/compsci2026
```

A **compiler** is a program that converts human-readable *source code* into machine-readable *object code* (often called an executable). Once compiled, the executable is typically very fast to run.

Compile the C program (GNU C Compiler):

```bash
gcc hello.c -o hello_c
```

Compile the Fortran program (GNU Fortran compiler):

```bash
gfortran hello.f90 -o hello_fortran
```

List your directory:

```bash
ls -l
```

Notice that the *source* files usually have permissions like `-rw-rw-r--` whereas the compiled executables have execute bits like `-rwxrwxr-x`.

> **Note:** Python is typically run via the CPython interpreter (written largely in C). Python source is compiled to bytecode and executed by a virtual machine; the key point here is that your Python loop runs inside an interpreter/runtime rather than as a standalone native executable.

Run all three programs:

```bash
./hello_c
./hello_fortran
python3 hello.py
```

### Deliverables (Section B)

- Names of the three source files
- The compile commands you used
- The output of running each program


---
## Section C: Performance Comparison on a Large Vector



In this section you will run a simple numerical workload in C, Fortran, and Python, and compare performance using `/usr/bin/time -v`.

Here is the same program, written in all 3 languages:

First in C:

```C
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

int main(void) {
    const int n = 10000000;  // 10 million samples
    double *signal;
    double sumsq = 0.0;
    double rms;

    // Allocate memory
    signal = (double *)malloc(n * sizeof(double));
    if (signal == NULL) {
        fprintf(stderr, "Memory allocation failed\n");
        return 1;
    }

    // Fill signal
    for (int i = 0; i < n; i++) {
        signal[i] = sin(0.001 * (i + 1));
    }

    // Compute RMS
    for (int i = 0; i < n; i++) {
        sumsq += signal[i] * signal[i];
    }

    rms = sqrt(sumsq / n);

    printf("Samples: %d\n", n);
    printf("RMS amplitude: %.6f\n", rms);

    free(signal);
    return 0;
}
```

In Fortran90:

```Fortran
program rms_large
  implicit none

  integer, parameter :: n = 10000000
  integer :: i
  real(8), allocatable :: signal(:)
  real(8) :: sumsq, rms

  allocate(signal(n))

  ! Fill signal
  do i = 1, n
     signal(i) = sin(0.001d0 * dble(i))
  end do

  ! Compute RMS
  sumsq = 0.0d0
  do i = 1, n
     sumsq = sumsq + signal(i) * signal(i)
  end do

  rms = sqrt(sumsq / dble(n))

  print *, "Samples:", n
  print *, "RMS amplitude:", rms

  deallocate(signal)
end program rms_large
```
In Python (without numpy):
```Python
import math

n = 10_000_000
signal = [0.0] * n  # allocate list

# Fill signal
for i in range(n):
    signal[i] = math.sin(0.001 * (i + 1))

# Compute RMS
sumsq = 0.0
for x in signal:
    sumsq += x * x

rms = math.sqrt(sumsq / n)

print("Samples:", n)
print("RMS amplitude:", rms)
```

### C1. Copy the provided files into your directory

On `newton`, copy these files from:

```bash
/home/thompsong/compsci2026/week1
```

into your own directory:

- `rms_large.c`
- `rms_large.f90`
- `rms_large.py`

Commands (run on `newton`):

```bash
cp /home/thompsong/compsci2026/week1/rms_large.c  ~/compsci2026/
cp /home/thompsong/compsci2026/week1/rms_large.f90 ~/compsci2026/
cp /home/thompsong/compsci2026/week1/rms_large.py ~/compsci2026/
```

Then:

```bash
cd ~/compsci2026
ls -l rms_large.*
```



### C2. Compile C and Fortran with optimization

Compile C:

```bash
gcc -O2 rms_large.c -o rms_c -lm
```

Compile Fortran:

```bash
gfortran -O2 rms_large.f90 -o rms_fortran
```

### C3. Time each run

Time the compiled C executable:

```bash
/usr/bin/time -v ./rms_c
```

Time the compiled Fortran executable:

```bash
/usr/bin/time -v ./rms_fortran
```

Time the Python program:

```bash
/usr/bin/time -v python3 rms_large.py
```

### C4. Compare these quantities across all three runs

- **User time**
- **System time**
- **Maximum resident set size** (memory)

Tip: In the `/usr/bin/time -v` output, you can usually find lines like:
- `User time (seconds): ...`
- `System time (seconds): ...`
- `Maximum resident set size (kbytes): ...`



### C5. Reflection paragraph (≈150–200 words)

Write one paragraph addressing:

- Why the runtimes differ
- Why Python is slower in this case
- Why NumPy exists
- Why C and Fortran are still used in scientific computing

### Deliverables (Section C)

- Timing output (or summarized values) for all three programs
- Your comparison paragraph


---
## Notes & Expectations



- You are **not** graded on C or Fortran style.
- You are graded on:
  - successfully compiling and running programs
  - using Unix tools correctly
  - interpreting performance results
- This assignment is intentionally designed to be completed even if you have never written C or Fortran before.

### Why this assignment works well

Pedagogically, this does a lot in Week 1:

- Normalizes remote-first computing
- Introduces compilers vs interpreters
- Makes performance differences visible
- Creates a natural bridge to:
  - Python + NumPy
  - ObsPy internals
  - HPC workflows
- Sets the tone that tools (including GenAI) are part of scientific practice


In [ ]:

# Optional: record your timing summary here after running commands on newton.
# Fill in the numbers you observed.

results = {
    "C": {"user_s": None, "sys_s": None, "max_rss_kb": None},
    "Fortran": {"user_s": None, "sys_s": None, "max_rss_kb": None},
    "Python": {"user_s": None, "sys_s": None, "max_rss_kb": None},
}

results


In [ ]:

# Optional: write your reflection paragraph here.

reflection = """<Write ~150–200 words here.>
"""

print(reflection)
